## Import Modules for Image processing

Description of all image processing relevant library imports:

    pip install torchmetrics -q

import pandas as pd import numpy as np import os import re import copy import time from PIL import Image import seaborn as sns import matplotlib.pyplot as plt from collections.


from torch.utils.data import Dataset, DataLoader from torchvision import transforms from PIL import Image import torch import torch.nn as nn from torchvision import models from transformers, from torchmetrics.classification import MulticlassF1Score import torch.nn.functional as F

from tensorflow.keras.preprocessing.image import load_img, array_to_img import warnings %matplotlib inline warnings.filterwarnings('ignore')


In [1]:
pip install torchmetrics -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import numpy as np
import os
import re
import copy
import time
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models

from torchmetrics.classification import MulticlassF1Score
import torch.nn.functional as F

from tensorflow.keras.preprocessing.image import load_img, array_to_img
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [ ]:
torch.cuda.set_device(torch.cuda.current_device()) if torch.cuda.is_available() else print("CUDA is not available. Using CPU.")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load Dataset

In [ ]:
# !unzip -q "/your-path/images.zip"

In [ ]:
# create image path to access images
train['image_path'] = train.apply(lambda x: f"images/image_train/image_{x['imageid']}_product_{x['productid']}.jpg", axis=1)
train.head()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
train['prdtypecode'].value_counts()

In [ ]:
# total output classes in the dataset
train['prdtypecode'].nunique()

## Data Preprocessing for Image-Model Training


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train['prdtypecode'] = le.fit_transform(train['prdtypecode'])

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42, stratify=train['prdtypecode'])

In [ ]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [ ]:
class RakutenDataset(Dataset):
    def __init__(self, df, transform=None, max_text_length=50):
        self.image_paths = df['image_path']
        

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        text = self.text_data[idx]
        label = self.labels[idx]

        # Load and transform the image
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)


        # One-hot encode the label
        # label_one_hot = F.one_hot(torch.tensor(label), num_classes=self.num_classes)

        return {
            'image': image,
            'text_tokens': text_tokens,
            'label': torch.tensor(label)
        }

In [ ]:
# Define transformations for training images
image_transform_train = transforms.Compose([
    transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0), ratio=(0.8, 1.2)),  # Resize to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust color jitter
    transforms.RandomRotation(degrees=15),  # Randomly rotate the image by up to 15 degrees
    transforms.RandomVerticalFlip(),  # Randomly flip the image vertically
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=0.1),  # Random affine transformations
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

# Define transformations for validation images
image_transform_val = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(), # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

In [ ]:
# Create an instance of the dataset
train_dataset = RakutenDataset(train_df, transform=image_transform_train)
val_dataset = RakutenDataset(val_df, transform=image_transform_val)

